In [1]:
# TEST CELL - Check if kernel can execute code at all
# This cell should run BEFORE cell 1 to verify kernel is working
import os
LOG_PATH = '/Volumes/SSanDisk/SpeechRec-German/.cursor/debug.log'
try:
    os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)
    with open(LOG_PATH, 'w') as f:  # 'w' to overwrite, not append
        import json, time
        f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"TEST","location":"notebook:test_cell","message":"TEST CELL EXECUTED - Kernel is working","data":{},"timestamp":int(time.time()*1000)}) + '\n')
        f.flush()
        os.fsync(f.fileno())
    print("✓ Test cell executed successfully - kernel is working")
except Exception as e:
    print(f"✗ Test cell failed: {e}")
    try:
        with open('/tmp/kernel_test_error.txt', 'w') as f:
            f.write(f"Test cell error: {e}\n")
            import traceback
            traceback.print_exc(file=f)
    except: pass


✓ Test cell executed successfully - kernel is working


In [2]:
# CRITICAL: Safe torch import - must run BEFORE cell 2
# This cell safely imports torch and enables MPS (GPU) if available
import os
import sys

# Enable MPS (don't disable it)
# Remove any MPS-disabling environment variables if they were set
os.environ.pop('PYTORCH_MPS_DISABLE', None)
os.environ.pop('PYTORCH_ENABLE_MPS_FALLBACK', None)
os.environ.pop('PYTORCH_MPS_HIGH_WATERMARK_RATIO', None)

# Try to import torch safely
try:
    import torch
    
    # Make torch available globally
    sys.modules['torch'] = torch
    
    # Check MPS availability
    mps_available = torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else False
    mps_built = torch.backends.mps.is_built() if hasattr(torch.backends.mps, 'is_built') else False
    
    print(f"✓ torch {torch.__version__} imported successfully")
    
    if mps_available and mps_built:
        # Test MPS with a simple operation
        try:
            test_tensor = torch.randn(2, 2).to("mps")
            _ = test_tensor + 1
            print(f"✓ MPS (GPU) is available and working - will use GPU for training")
        except Exception as e:
            print(f"⚠ MPS is available but test failed: {e}")
            print("  Will fall back to CPU if needed")
    else:
        print(f"⚠ MPS (GPU) is not available - will use CPU for training")
    
except Exception as e:
    print(f"✗ ERROR importing torch: {e}")
    print("\nPossible solutions:")
    print("1. Update PyTorch: pip install --upgrade torch")
    print("2. Install openmp: conda install -c intel openmp")
    print("3. Use a different Python environment")
    raise


✓ torch 2.9.1 imported successfully
✓ MPS (GPU) is available and working - will use GPU for training


# Spectrogram Models Training

Training models on spectrograms:
- Model 1: ResNet on spectrograms
- Model 2: Vision Transformer (ViT) on spectrograms

In [3]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import json
import gc
import h5py
import os

# Add project root to path
# Determine project root (parent of notebooks directory)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name in ['notebooks', 'b-p_first_experiments'] else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

from models.spectrogram.resnet_spectrogram import resnet18_spectrogram
from models.spectrogram.vit_spectrogram import VisionTransformerSpectrogram
from utils.training_utils import train_model, evaluate_model
from utils.data_loader import SpectrogramDataset, load_metadata
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np

# Device setup

# MPS has known issues with BatchNorm and some operations - use CPU for stability
# Uncomment the line below to force CPU if MPS causes crashes
FORCE_CPU = False  # Set to True to force CPU, False to use GPU (MPS on macOS)

if not FORCE_CPU and torch.backends.mps.is_available():
    # Test MPS with a simple operation to check if it works
    try:
        test_tensor = torch.randn(2, 2).to("mps")
        _ = test_tensor + 1
        device = torch.device("mps")
        print(f"Using MPS device (tested)")
    except Exception as e:
        print(f"MPS test failed, falling back to CPU: {e}")
        device = torch.device("cpu")
elif not FORCE_CPU and torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device")
else:
    device = torch.device("cpu")
    print(f"Using CPU device")

# Load data step by step to avoid memory issues
print("\nLoading data...")
try:
    # Load metadata first (without loading spectrograms into memory)
    df, feature_cols, feature_scaler, class_weights_dict = load_metadata(PROJECT_ROOT, check_spectrograms=True)
    print(f"DataFrame loaded: {len(df)} samples")
    
    # Load spectrograms lazily using H5 file reference
    DATA_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dataset'
    SPECTROGRAMS_FILE = DATA_DIR / 'features' / 'spectrograms.h5'
    
    # Open H5 file and keep reference (lazy loading)
    h5_file = h5py.File(SPECTROGRAMS_FILE, 'r')
    print(f"Spectrograms file opened: {len(h5_file.keys())} entries")
    
    # Create a wrapper dict that loads from H5 on demand
    class LazySpectrogramDict:
        def __init__(self, h5_file):
            self.h5_file = h5_file
            self._cache = {}
            self._cache_size = 0
            self._max_cache_size = 1000  # Cache up to 1000 spectrograms
        
        def __getitem__(self, key):
            if key in self._cache:
                return self._cache[key]
            
            if key in self.h5_file:
                spectrogram = self.h5_file[key][:].astype(np.float32)
                
                # Simple LRU cache: if cache is full, remove oldest
                if self._cache_size >= self._max_cache_size:
                    # Remove first item (simple FIFO)
                    first_key = next(iter(self._cache))
                    del self._cache[first_key]
                    self._cache_size -= 1
                
                self._cache[key] = spectrogram
                self._cache_size += 1
                return spectrogram
            else:
                raise KeyError(f"Spectrogram {key} not found")
        
        def __contains__(self, key):
            return key in self.h5_file
        
        def keys(self):
            return self.h5_file.keys()
        
        def close(self):
            self.h5_file.close()
            self._cache.clear()
    
    spectrograms_dict = LazySpectrogramDict(h5_file)
    
    # Filter df to only include samples with spectrograms
    df['has_spectrogram'] = df['phoneme_id'].isin(spectrograms_dict.keys())
    df = df[df['has_spectrogram']].copy().reset_index(drop=True)
    print(f"Filtered to {len(df)} samples with spectrograms")
    
    # Create datasets only for spectrograms (not all types)
    train_ds = SpectrogramDataset(df, spectrograms_dict, split='train')
    val_ds = SpectrogramDataset(df, spectrograms_dict, split='val')
    test_ds = SpectrogramDataset(df, spectrograms_dict, split='test')
    
    print(f"Datasets created: Train={len(train_ds)}, Val={len(val_ds)}, Test={len(test_ds)}")
    
    # Create sampler for training
    train_labels = df[df['split'] == 'train']['class_encoded'].values
    class_weights = np.array([
        class_weights_dict.get(str(label), class_weights_dict.get(label, 1.0)) 
        for label in train_labels
    ])
    sampler = WeightedRandomSampler(
        weights=class_weights,
        num_samples=len(class_weights),
        replacement=True
    )
    
    # Create dataloaders with smaller batch size and num_workers=0
    batch_size = 32  # Reduced from 64 to avoid memory issues
    train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, num_workers=0, pin_memory=False)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)
    
    OUTPUT_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dl_models'
    
    # Determine number of classes from data
    num_classes = df['class_encoded'].nunique()
    unique_classes = sorted(df['class_encoded'].unique())
    print(f"\n{'='*60}")
    print(f"Class Information:")
    print(f"{'='*60}")
    print(f"Number of classes: {num_classes}")
    print(f"Unique class labels: {unique_classes}")
    print(f"Class distribution:")
    print(df['class_encoded'].value_counts().sort_index())
    print(f"{'='*60}")
    
    print(f"\nData loaded successfully!")
    print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}, Test batches: {len(test_loader)}")
    print(f"Batch size: {batch_size}")
    
    # Clean up
    gc.collect()
    
except Exception as e:
    print(f"Error loading data: {e}")
    import traceback
    traceback.print_exc()
    raise

Using MPS device (tested)

Loading data...
Columns in df_phonemes: ['phoneme_id', 'utterance_id', 'phoneme', 'class', 'start_ms', 'end_ms', 'duration_ms', 'audio_path']
Columns in df_features: ['energy_rms', 'energy_rms_std', 'energy_zcr', 'energy_zcr_std', 'spectral_centroid', 'spectral_centroid_std', 'spectral_rolloff', 'spectral_rolloff_std', 'spectral_bandwidth', 'spectral_bandwidth_std', 'formant_f1', 'formant_f2', 'formant_f3', 'formant_f4', 'formant_f1_std', 'formant_f2_std', 'formant_f3_std', 'formant_f4_std', 'spectral_flatness', 'harmonic_noise_ratio', 'zcr_mean', 'energy_cv', 'phoneme_id', 'class', 'duration_ms', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_std_0', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 

## Model 1: ResNet on Spectrograms

In [4]:
# NOTE: Do NOT close H5 file here - it's needed for training!
# The file will be closed after all models are trained

In [5]:
try:
    # Initialize model with correct number of classes
    model1 = resnet18_spectrogram(num_classes=num_classes)
    model1 = model1.to(device)
    print(f"Model 1 (ResNet) parameters: {sum(p.numel() for p in model1.parameters()):,}")
    print(f"Model configured for {num_classes} classes")

    # Loss and optimizer - create class weights dynamically
    class_weights_list = [
        class_weights_dict.get(str(i), class_weights_dict.get(i, 1.0))
        for i in range(num_classes)
    ]
    class_weights = torch.tensor(class_weights_list, dtype=torch.float32).to(device)
    print(f"Class weights: {class_weights.cpu().numpy()}")
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    # Train
    save_dir = OUTPUT_DIR / 'spectrogram_models' / 'resnet_spectrogram'
    save_dir.mkdir(parents=True, exist_ok=True)

    print("\nTraining Model 1 (ResNet)...")
    history1, best_epoch1 = train_model(
        model1, train_loader, val_loader, criterion, optimizer, scheduler,
        device, num_epochs=50, save_dir=save_dir, model_name='resnet_spectrogram', early_stopping_patience=10
    )

    # Evaluate on test set
    print("\nEvaluating Model 1 on test set...")
    checkpoint = torch.load(save_dir / 'best_model.pt', map_location=device)
    model1.load_state_dict(checkpoint['model_state_dict'])
    test_metrics1, _, _, _ = evaluate_model(model1, test_loader, criterion, device)

    # Save test metrics
    with open(save_dir / 'test_metrics.json', 'w') as f:
        json.dump(test_metrics1, f, indent=2)

    print(f"\n{'='*60}")
    print(f"Model 1 (ResNet) Test Results:")
    print(f"{'='*60}")
    print(f"Accuracy: {test_metrics1['accuracy']:.4f}")
    print(f"F1: {test_metrics1['f1']:.4f}")
    print(f"ROC-AUC: {test_metrics1['roc_auc']:.4f}")
    print(f"Precision (/b/): {test_metrics1['precision_b']:.4f}")
    print(f"Precision (/p/): {test_metrics1['precision_p']:.4f}")
    print(f"Recall (/b/): {test_metrics1['recall_b']:.4f}")
    print(f"Recall (/p/): {test_metrics1['recall_p']:.4f}")
    
    # Clean up model 1
    del model1, optimizer, scheduler, criterion
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    elif torch.cuda.is_available():
        torch.cuda.empty_cache()
    
except Exception as e:
    print(f"Error in Model 1 training: {e}")
    import traceback
    traceback.print_exc()
    raise

Model 1 (ResNet) parameters: 11,168,706
Model configured for 2 classes
Class weights: [0.7156037 1.6595349]

Training Model 1 (ResNet)...

Epoch 1/50
--------------------------------------------------


Train Loss: 0.2945, Train Acc: 0.8491
Val Loss: 0.2544, Val Acc: 0.8960
Val F1: 0.8975, Val ROC-AUC: 0.9612
✓ New best model saved! (F1: 0.8975)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.2350, Train Acc: 0.8867
Val Loss: 0.2503, Val Acc: 0.9040
Val F1: 0.9052, Val ROC-AUC: 0.9622
✓ New best model saved! (F1: 0.9052)

Epoch 3/50
--------------------------------------------------


Train Loss: 0.2135, Train Acc: 0.8994
Val Loss: 0.2190, Val Acc: 0.9123
Val F1: 0.9134, Val ROC-AUC: 0.9705
✓ New best model saved! (F1: 0.9134)

Epoch 4/50
--------------------------------------------------


Train Loss: 0.1986, Train Acc: 0.9052
Val Loss: 0.3682, Val Acc: 0.8240
Val F1: 0.8310, Val ROC-AUC: 0.9565

Epoch 5/50
--------------------------------------------------


...

Train Loss: 0.0021, Train Acc: 0.9993
Val Loss: 0.4508, Val Acc: 0.9357
Val F1: 0.9359, Val ROC-AUC: 0.9797

Epoch 26/50
--------------------------------------------------


Train Loss: 0.0020, Train Acc: 0.9992
Val Loss: 0.4937, Val Acc: 0.9359
Val F1: 0.9358, Val ROC-AUC: 0.9796

Epoch 27/50
--------------------------------------------------


Train Loss: 0.0028, Train Acc: 0.9990
Val Loss: 0.4940, Val Acc: 0.9368
Val F1: 0.9368, Val ROC-AUC: 0.9797

Epoch 28/50
--------------------------------------------------


Train Loss: 0.0012, Train Acc: 0.9995
Val Loss: 0.4919, Val Acc: 0.9342
Val F1: 0.9340, Val ROC-AUC: 0.9797

Early stopping at epoch 28
Best F1: 0.9400 at epoch 18

Evaluating Model 1 on test set...



Model 1 (ResNet) Test Results:
Accuracy: 0.9331
F1: 0.9329
ROC-AUC: 0.9760
Precision (/b/): 0.9490
Precision (/p/): 0.8953
Recall (/b/): 0.9556
Recall (/p/): 0.8808


## Model 2: Vision Transformer on Spectrograms

In [6]:
# NOTE: Cleanup will be done after Model 2 training (see last cell)


In [ ]:
try:
    # Initialize model with correct number of classes
    model2 = VisionTransformerSpectrogram(
        img_size=(128, 7),
        patch_size=(16, 1),
        embed_dim=128,
        depth=6,
        num_heads=8,
        num_classes=num_classes
    )
    model2 = model2.to(device)
    print(f"Model 2 (ViT) parameters: {sum(p.numel() for p in model2.parameters()):,}")
    print(f"Model configured for {num_classes} classes")

    # Loss and optimizer - create class weights dynamically
    class_weights_list = [
        class_weights_dict.get(str(i), class_weights_dict.get(i, 1.0))
        for i in range(num_classes)
    ]
    class_weights = torch.tensor(class_weights_list, dtype=torch.float32).to(device)
    print(f"Class weights: {class_weights.cpu().numpy()}")
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.AdamW(model2.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

    # Train
    save_dir = OUTPUT_DIR / 'spectrogram_models' / 'vit_spectrogram'
    save_dir.mkdir(parents=True, exist_ok=True)

    print("\nTraining Model 2 (ViT)...")
    history2, best_epoch2 = train_model(
        model2, train_loader, val_loader, criterion, optimizer, scheduler,
        device, num_epochs=50, save_dir=save_dir, model_name='vit_spectrogram', early_stopping_patience=10
    )

    # Evaluate on test set
    print("\nEvaluating Model 2 on test set...")
    checkpoint = torch.load(save_dir / 'best_model.pt', map_location=device)
    model2.load_state_dict(checkpoint['model_state_dict'])
    test_metrics2, _, _, _ = evaluate_model(model2, test_loader, criterion, device)

    # Save test metrics
    with open(save_dir / 'test_metrics.json', 'w') as f:
        json.dump(test_metrics2, f, indent=2)

    print(f"\n{'='*60}")
    print(f"Model 2 (ViT) Test Results:")
    print(f"{'='*60}")
    print(f"Accuracy: {test_metrics2['accuracy']:.4f}")
    print(f"F1: {test_metrics2['f1']:.4f}")
    print(f"ROC-AUC: {test_metrics2['roc_auc']:.4f}")
    print(f"Precision (/b/): {test_metrics2['precision_b']:.4f}")
    print(f"Precision (/p/): {test_metrics2['precision_p']:.4f}")
    print(f"Recall (/b/): {test_metrics2['recall_b']:.4f}")
    print(f"Recall (/p/): {test_metrics2['recall_p']:.4f}")
    
    # Clean up model 2
    del model2, optimizer, scheduler, criterion
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    elif torch.cuda.is_available():
        torch.cuda.empty_cache()
    
except Exception as e:
    print(f"Error in Model 2 training: {e}")
    import traceback
    traceback.print_exc()
    raise

Model 2 (ViT) parameters: 1,199,746
Model configured for 2 classes
Class weights: [0.7156037 1.6595349]

Training Model 2 (ViT)...

Epoch 1/50
--------------------------------------------------


Train Loss: 0.4911, Train Acc: 0.7004
Val Loss: 0.5517, Val Acc: 0.6862
Val F1: 0.6989, Val ROC-AUC: 0.8179
✓ New best model saved! (F1: 0.6989)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.4729, Train Acc: 0.7203
Val Loss: 0.6113, Val Acc: 0.6341
Val F1: 0.6433, Val ROC-AUC: 0.8270

Epoch 3/50
--------------------------------------------------


Train Loss: 0.4570, Train Acc: 0.7249
Val Loss: 0.5806, Val Acc: 0.6201
Val F1: 0.6285, Val ROC-AUC: 0.8296

Epoch 4/50
--------------------------------------------------


Train Loss: 0.4095, Train Acc: 0.7555
Val Loss: 0.4140, Val Acc: 0.7747
Val F1: 0.7839, Val ROC-AUC: 0.9139
✓ New best model saved! (F1: 0.7839)

Epoch 5/50
--------------------------------------------------


...

Train Loss: 0.1802, Train Acc: 0.9133
Val Loss: 0.2278, Val Acc: 0.8948
Val F1: 0.8978, Val ROC-AUC: 0.9734

Epoch 32/50
--------------------------------------------------


Train Loss: 0.1793, Train Acc: 0.9138
Val Loss: 0.2103, Val Acc: 0.9106
Val F1: 0.9125, Val ROC-AUC: 0.9738

Epoch 33/50
--------------------------------------------------


Train Loss: 0.1804, Train Acc: 0.9134
Val Loss: 0.2131, Val Acc: 0.9098
Val F1: 0.9118, Val ROC-AUC: 0.9737

Epoch 34/50
--------------------------------------------------


Train Loss: 0.1771, Train Acc: 0.9124
Val Loss: 0.2169, Val Acc: 0.9072
Val F1: 0.9092, Val ROC-AUC: 0.9729

Early stopping at epoch 34
Best F1: 0.9137 at epoch 24

Evaluating Model 2 on test set...



Model 2 (ViT) Test Results:
Accuracy: 0.9106
F1: 0.9122
ROC-AUC: 0.9691
Precision (/b/): 0.9657
Precision (/p/): 0.8064
Recall (/b/): 0.9042
Recall (/p/): 0.9255


In [ ]:
# Final cleanup: Close H5 file after all training is complete
if 'spectrograms_dict' in locals() and hasattr(spectrograms_dict, 'close'):
    spectrograms_dict.close()
    print("H5 file closed successfully")
    
# Final memory cleanup
gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
elif torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Memory cleaned up")
print("\nAll models trained successfully!")


H5 file closed successfully
Memory cleaned up

All models trained successfully!
